In [6]:
import tensorflow as tf
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

Dataset= pd.read_csv("Training.csv")


Dataset=Dataset.dropna() # remove null values
Dataset=Dataset.drop("Id",axis=1) # remove sample information
#print(Dataset.value_counts(Dataset["Class"]))
Labels=Dataset[["Class"]].copy() # generate separate class dataframe
Dataset=Dataset.drop("Class",axis=1) # drop class from feature dataframe
mapping = {'A': 1, 'B': 0} # change object from str to int 
Dataset['EJ'] = Dataset['EJ'].replace(mapping)
#print(Dataset.shape)
Dataset_train, Dataset_test, Labels_train, Labels_test = train_test_split(Dataset, Labels, test_size=0.5, random_state=42)
#print(type(Dataset_train))
sm = SMOTE(random_state=42)
AltDataset, AltLabels = sm.fit_resample(Dataset_train, Labels_train)
#print(AltLabels.value_counts(AltLabels["Class"]))
AltDataset = tf.convert_to_tensor(AltDataset.values, dtype=tf.float32)
AltLabels=tf.convert_to_tensor(AltLabels.values, dtype=tf.float32)
print(len(Labels_train))
DiseaseModel=keras.Sequential([layers.Dense(300,activation="relu"),layers.Dense(1,activation="sigmoid")])
DiseaseModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
DiseaseModel.fit(AltDataset,AltLabels,epochs=30,batch_size=150)
print("I will determine the accuracy of my test data.")
Evaluations=DiseaseModel.evaluate(Dataset_test,Labels_test)

274
Epoch 1/30
3/3 [==============================] - 0s 4ms/step - loss: 622.6439 - accuracy: 0.5113
Epoch 2/30
3/3 [==============================] - 0s 5ms/step - loss: 212.9191 - accuracy: 0.5249
Epoch 3/30
3/3 [==============================] - 0s 4ms/step - loss: 299.9429 - accuracy: 0.5023
Epoch 4/30
3/3 [==============================] - 0s 4ms/step - loss: 147.0656 - accuracy: 0.5792
Epoch 5/30
3/3 [==============================] - 0s 3ms/step - loss: 175.1088 - accuracy: 0.6018
Epoch 6/30
3/3 [==============================] - 0s 4ms/step - loss: 167.3060 - accuracy: 0.6018
Epoch 7/30
3/3 [==============================] - 0s 5ms/step - loss: 85.1299 - accuracy: 0.6041
Epoch 8/30
3/3 [==============================] - 0s 4ms/step - loss: 84.5457 - accuracy: 0.5814
Epoch 9/30
3/3 [==============================] - 0s 4ms/step - loss: 55.4053 - accuracy: 0.6244
Epoch 10/30
3/3 [==============================] - 0s 3ms/step - loss: 57.4775 - accuracy: 0.6290
Epoch 11/30
3/3 [==

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def create_model():
    model = Sequential([Dense(140, activation="relu"),Dense(30, activation="swish"), Dense(1, activation="sigmoid")])
    return model

loss_function = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

population_size=100


population=[[create_model() for amount in range(2)] for size in range(population_size)]

optimizer = tf.keras.optimizers.legacy.Adam()


for generation in range(min(50, len(AltDataset))):  # For each generation
    losses = []
    for Individual in population:
        Individual_Loss = 0
        sample, label = AltDataset[generation], AltLabels[generation]
        for model in Individual:
            with tf.GradientTape() as tape:
                prediction = model(sample[None, ...], training=True)
                loss = loss_function(label[None, ...], prediction)

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # Use the same optimizer instance

            Individual_Loss += loss    
        losses.append(Individual_Loss/2)


    top_individuals_indices= (np.argsort(losses))[0:int((len(population)/2)+1)]
    #The following line identifies the positions of the individuals (models) 
    # with the lowest losses in the population. Specifically, it selects just over 
    # half of the population, ranking them by their loss values. This subset of 
    # indices can be used to select the top-performing individuals for further 
    # genetic operations such as crossover and mutation, thereby ensuring that 
    # the traits of the best-performing models are carried over to the next generation.
    
    
    def generate_haploid_cluster(individual, cluster_size): 
        # This function takes an individual organism containing two models (analogous to paired chromosomes) 
        # and generates a cluster of gametes (like sperm), each gamete is a single model which is a combination of the two models (analogous to process of meiosis).

        haploid_cluster = [] # List to store the generated gametes

        for gamete_index in range(cluster_size): # Loop to create the desired number of gametes
            paired_chromosomes = individual # Retrieve the two models representing the individual's paired chromosomes
            gamete = create_model() # Create a new model with the same architecture
            gamete.build((None, sample.shape[0])) # Initialize the weights of the gamete model based on the anticipated input size

            # Loop through the weight tensors in the gamete model
            for weight_index in range(len(gamete.weights)): 
                # Create a mask of random 0s and 1s to determine which genetic material to inherit from each paired chromosome
                mask = np.random.choice([0, 1], size=gamete.weights[weight_index].shape.as_list())
                
                # Use the mask to combine genetic material from both paired chromosomes, element by element
                combined_weights = np.where(mask, paired_chromosomes[0].weights[weight_index].numpy(), paired_chromosomes[1].weights[weight_index].numpy())

                # Assign the combined genetic material to the gamete model
                gamete.weights[weight_index].assign(combined_weights)

            haploid_cluster.append(gamete) # Add the gamete to the cluster

        return haploid_cluster # Return the cluster of gametes
    

    def gamete_selection(haploid_cluster, generation):
        """
        Selects the best gamete (model) from a haploid cluster based on its performance
        on a given subset of the dataset.

        Parameters:
            haploid_cluster (list): A cluster of gametes (models) to be evaluated.
            generation (int): Specifies the number of samples to be used from the beginning of the dataset.

        Returns:
            best_gamete (model): The gamete (model) with the lowest loss on the specified subset of the dataset.
        """
        best_gamete_fitness_score = float("inf")
        best_gamete = None
        
        # Selecting samples from the dataset based on the current generation
        sample, label = AltDataset[0:generation], AltLabels[0:generation]

        for gamete in haploid_cluster:
            # Calculating the fitness score (loss) for each gamete using the specified samples
            with tf.GradientTape() as tape:
                prediction = gamete(sample, training=True)  # Forward pass
                loss_fitness_score = loss_function(label, prediction)  # Compute the loss

            # Updating the best gamete if the current one has a lower loss
            if loss_fitness_score < best_gamete_fitness_score:
                best_gamete = gamete
                best_gamete_fitness_score = loss_fitness_score
                
        return best_gamete

    def offspring_fertilisation(parent_one_gamete,parent_two_gamete): #Two gametes come together to form a whole individual with a complete set of chromosomes (two models)
        offspring=[parent_one_gamete,parent_two_gamete]
        return(offspring)

       # Create the next generation
    count = 0

    next_generation = []  # Make sure to initialize the next_generation list
    for i in range(len(top_individuals_indices) - 1):  # Pairing individuals
        # Only proceed with even counts
        if count % 2 != 0:
            count += 1
            continue

        # Generate haploid clusters for parent individuals
        haploid_cluster_parent_one = generate_haploid_cluster(population[top_individuals_indices[count]], cluster_size=10)
        haploid_cluster_parent_two = generate_haploid_cluster(population[top_individuals_indices[count + 1]], cluster_size=10)

        # Select the best gametes from each cluster
        parent_one_gamete = gamete_selection(haploid_cluster_parent_one, generation)
        parent_two_gamete = gamete_selection(haploid_cluster_parent_two, generation)

        # Fertilize the selected gametes to create offspring
        offspring1 = offspring_fertilisation(parent_one_gamete, parent_two_gamete)
        offspring2= offspring_fertilisation(parent_one_gamete, parent_two_gamete)
        # Add the offspring to the next generation
        next_generation.append(offspring1)
        next_generation.append(offspring2)
        count += 1
    population=next_generation
    print(population)


    


In [14]:
print(population)
from tensorflow.keras.metrics import BinaryAccuracy

for i, individual in enumerate(population):  
      specificity=0
      sensitivity=0
      precision_class_one=0
      Negative_Predictive_Value=0
      accuracy=0
      for model in individual:
            y_pred = model.predict(Dataset_test)  # Assuming Dataset_test is your test data
            y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to class labels (0 or 1)

            conf_matrix = np.zeros((2, 2), dtype=int)
            conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
            conf_matrix[0, 1] = np.sum((Labels_test.values == 0) & (y_pred == 1))  # False Positives (FP)
            conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
            conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

            # Extract true negatives (TN), false positives (FP), true positives (TP), and false negatives (FN) from the confusion matrix
            TN, FP, TP, FN = conf_matrix[0, 0], conf_matrix[0, 1], conf_matrix[1, 1], conf_matrix[1, 0]

            # Calculate specificity, sensitivity, precision, NPV, and accuracy
            specificity = specificity+(TN / (TN + FP))
            sensitivity = sensitivity+(TP / (TP + FN))
            precision_class_one = precision_class_one+(TP / (TP + FP))
            Negative_Predictive_Value =Negative_Predictive_Value+( TN / (TN + FN))
            accuracy =accuracy+((TP + TN) / (TP + TN + FP + FN))

      # Print the metrics
      #"""
      print(f"Individual {i+1}")
      print(f"Sensitivity: {sensitivity/2 * 100:.2f}%,",
            f"Specificity: {specificity/2 * 100:.2f}%,",
            f"Accuracy: {accuracy/2 * 100:.2f}%,",
            f"Negative Predictive Value: {Negative_Predictive_Value/2 * 100:.2f}%,",
            f"Precision (Class One): {precision_class_one/2 * 100:.2f}%")
      print(f"True Negatives (TN): {TN},",
            f"False Positives (FP): {FP},",
            f"True Positives (TP): {TP},",
            f"False Negatives (FN): {FN}")
      #"""


Proportion = Labels_test["Class"].value_counts()
print("By random chance the precision would be {:.2f}%".format(Proportion[1] / (Proportion[0] + Proportion[1]) * 100))


[]
By random chance the precision would be 17.88%


In [ ]:
import numpy as np

# Assuming y_true and y_pred are numpy arrays representing the true labels and predicted labels, respectively.
# 1 represents the positive class, and 0 represents the negative class.
y_pred=model.predict(Dataset_test)
# Compute the confusion matrix
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(f"Specificity: {specificity * 100:.2f}%",
      f"Sensitivity: {sensitivity * 100:.2f}%",
      f"Accuracy: {accuracy * 100:.2f}%",
      f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%",
      f"Precision (Class One): {precision_class_one * 100:.2f}%")
print(f"True Negatives (TN): {TN}",
      f"False Positives (FP): {FP}",
      f"True Positives (TP): {TP}",
      f"False Negatives (FN): {FN}")


In [ ]:
import tensorflow as tf

# Assuming `model` is your trained model
# Assuming `X_test` is your test data and `y_test` are your test labels
y_pred = DiseaseModel.predict(Dataset_test)  # assuming x_test is your test data
y_pred = (y_pred > 0.5).astype(int)
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(sensitivity*100,specificity*100,accuracy*100,Negative_Predictive_Value*100,precision_class_one*100)
print(TN,FP,TP,FN)
#Sensitivity: 44.90%, Specificity: 82.22%, Accuracy: 75.55%, Negative Predictive Value: 87.26%, Precision (Class One): 35.48%
